In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import timeit

In [2]:
pos = np.load('pos_train_16.npy')
print(pos.shape)
pos_val = np.load('position_wheel_test.npy')
pos_val=pos_val.reshape(3852,24,3)
print(pos_val.shape)

(218, 321, 24, 3)
(3852, 24, 3)


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
class TrajectoryDataset(Dataset):
    def __init__(self, data_file):
        self.data = data_file
        self.num_frames = self.data.shape[0]
        
    def __len__(self):
        return self.num_frames - 15
    
    def __getitem__(self, idx):

        # Get the input-output pair for the given index
#         input_data = self.data[:, idx:idx+7, :, :].reshape(-1, 6*24*3)
#         output_data = self.data[:, idx+7:idx+8, :, :].reshape(-1, 24*3)
        input_data = self.data[idx:idx+7, :, :].reshape(-1, 7*24*3)
#         output_data = self.data[idx+7:idx+8, :, :].reshape(-1, 24*3)


        # Convert to PyTorch tensors and return
        input_tensor = torch.from_numpy(input_data).float()
#         output_tensor = torch.from_numpy(output_data).float()

        
        return input_tensor#, output_tensor

In [5]:
pos = pos.reshape(-1,24,3)
print(pos.shape)
a =TrajectoryDataset(pos)

(69978, 24, 3)


In [ ]:
# batch_size = 32
# # train_data = np.load('pos_train_16.npy')
# # train_dataset=train_data.reshape(-1,24,3)
# # print(train_dataset.shape)
# train_dataset = TrajectoryDataset(pos)

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# # Create a DataLoader for the validation dataset
# # val = np.load("position_wheel_test.npy")
# # val_data= val[:,:320,:]
# # val_dataset = val_data.reshape(-1,24,3)
# val_dataset = TrajectoryDataset(pos_val)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [6]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset

# create dummy data

data_tensor = pos

# create a TensorDataset from the data tensor
dataset = TrajectoryDataset(data_tensor)

# create a DataLoader with batch size 32
batch_size = 1
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# iterate over the data loader and print batch sizes
for i, batch in enumerate(train_dataloader):
    print(f"Batch {i}, Size: {batch.size()}")
    if i == 0:
        break



Batch 0, Size: torch.Size([1, 1, 504])


In [7]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(24*6*3, 1728)
        self.fc2 = nn.Linear(1728, 864)
        self.fc3 = nn.Linear(864, 432)
        self.fc4 = nn.Linear(432, 128)
        self.fc5 = nn.Linear(128, 24*3)

    def forward(self, x):
        x = self.flatten(x)
#         print(x.shape)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x.view(-1, 24, 3)

In [ ]:
# def test(model, data):
#     # Initialize the input sequence with the first 6 time steps from the data
#     input_seq = torch.Tensor(data[:, :6, :, :]).unsqueeze(0)  # shape: (1, 6, 24, 3)

#     # Initialize the output sequence with the first time step from the data
#     output_seq = torch.Tensor(data[:, 6:7, :, :])  # shape: (1, 1, 24, 3)

#     # Loop over the remaining time steps
#     for i in range(1, data.shape[1] - 6):
#         # Make a prediction for the next time step based on the input sequence
#         prediction = model(input_seq)

#         # Add the predicted time step to the output sequence
#         output_seq = torch.cat([output_seq, prediction], dim=1)

#         # Remove the first time step from the input sequence
#         input_seq = input_seq[:, 1:, :, :]

#         # Add the predicted time step to the input sequence
#         input_seq = torch.cat([input_seq, prediction], dim=1)

#     return output_seq


In [8]:
lr = 0.001
epochs = 10000

In [10]:
model = MLP()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda


MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=432, out_features=1728, bias=True)
  (fc2): Linear(in_features=1728, out_features=864, bias=True)
  (fc3): Linear(in_features=864, out_features=432, bias=True)
  (fc4): Linear(in_features=432, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=72, bias=True)
)

In [11]:
torch.cuda.is_available()

True

In [12]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [13]:
for batch in train_dataloader:
    print(batch.shape)

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

...

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1, 504])
torch.Size([1, 1

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 1000
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_input in train_dataloader:
        batch_input = torch.tensor(batch_input).to(device).float()
        batch_input = batch_input.reshape(7,24,3)
#         print('input of batch',batch_input.shape)
        input_data = batch_input[:6, :, :]
        input_data=input_data.reshape(1,432)
        ouput_data = batch_input[6:,:,:]
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        predicted_output = model(input_data)

        # Compute the loss
#         output_data=output_data.reshape(24,3)
        loss = criterion(predicted_output, batch_input[6:,:,:])

        # Backward pass and optimization step
        loss.backward()
        optimizer.step()

        # Update the running loss
        running_loss += loss.item() * batch_input.size(0)

    # Print the average training loss for the epoch
    train_epoch_loss = running_loss / len(dataset)
    print("Epoch {}: Training Loss = {:.4f}".format(epoch+1, train_epoch_loss))
    train_losses.append(train_epoch_loss)

#     # Compute the validation loss for the epoch
#     val_running_loss = 0.0
#     for batch in val_dataloader:
#         batch = batch.to(device)


#         with torch.no_grad():
#             predicted_output = model(batch[:6,:,:])
#             val_loss = criterion(predicted_output, batch[6:,:,:])
#             val_running_loss += val_loss.item() * batch_input.size(0)

#     val_epoch_loss = val_running_loss / len(val_dataset)
#     print("Epoch {}: Validation Loss = {:.4f}".format(epoch+1, val_epoch_loss))
#     val_losses.append(val_epoch_loss)


/tmp/ipykernel_17559/3770760824.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_input = torch.tensor(batch_input).to(device).float()


Epoch 1: Training Loss = 0.0007
Epoch 2: Training Loss = 0.0007
Epoch 3: Training Loss = 0.0006
Epoch 4: Training Loss = 0.0006
Epoch 5: Training Loss = 0.0006
Epoch 6: Training Loss = 0.0006
Epoch 7: Training Loss = 0.0006
Epoch 8: Training Loss = 0.0006
Epoch 9: Training Loss = 0.0006
Epoch 10: Training Loss = 0.0006
Epoch 11: Training Loss = 0.0006
Epoch 12: Training Loss = 0.0006
Epoch 13: Training Loss = 0.0006
Epoch 14: Training Loss = 0.0006
Epoch 15: Training Loss = 0.0006
Epoch 16: Training Loss = 0.0006
Epoch 17: Training Loss = 0.0006
Epoch 18: Training Loss = 0.0006
Epoch 19: Training Loss = 0.0007
Epoch 20: Training Loss = 0.0006
Epoch 21: Training Loss = 0.0006
Epoch 22: Training Loss = 0.0006
Epoch 23: Training Loss = 0.0006
Epoch 24: Training Loss = 0.0006
Epoch 25: Training Loss = 0.0006
Epoch 26: Training Loss = 0.0006
Epoch 27: Training Loss = 0.0006
Epoch 28: Training Loss = 0.0006
Epoch 29: Training Loss = 0.0007
Epoch 30: Training Loss = 0.0007
Epoch 31: Training 

Epoch 246: Training Loss = 0.0007
Epoch 247: Training Loss = 0.0007
Epoch 248: Training Loss = 0.0006
Epoch 249: Training Loss = 0.0006
Epoch 250: Training Loss = 0.0007
Epoch 251: Training Loss = 0.0006
Epoch 252: Training Loss = 0.0007
Epoch 253: Training Loss = 0.0007
Epoch 254: Training Loss = 0.0007
Epoch 255: Training Loss = 0.0007
Epoch 256: Training Loss = 0.0007
Epoch 257: Training Loss = 0.0006
Epoch 258: Training Loss = 0.0006
Epoch 259: Training Loss = 0.0006
Epoch 260: Training Loss = 0.0006
Epoch 261: Training Loss = 0.0006
Epoch 262: Training Loss = 0.0006
Epoch 263: Training Loss = 0.0006
Epoch 264: Training Loss = 0.0006
Epoch 265: Training Loss = 0.0006
Epoch 266: Training Loss = 0.0006
Epoch 267: Training Loss = 0.0006
Epoch 268: Training Loss = 0.0006
Epoch 269: Training Loss = 0.0006
Epoch 270: Training Loss = 0.0006
Epoch 271: Training Loss = 0.0006
Epoch 272: Training Loss = 0.0006
Epoch 273: Training Loss = 0.0006
Epoch 274: Training Loss = 0.0006
Epoch 275: Tra

KeyboardInterrupt: 

In [34]:
pos_val = np.load('position_wheel_test.npy')
print(pos_val.shape)
pos_val_1=pos_val[0,:,:]
pos_val_1=pos_val_1.reshape(321,24,3)
print(pos_val_1.shape)

(12, 321, 72)
(321, 24, 3)


In [35]:
initial_positions = pos_val_1[0:6,:,:]
print(initial_positions.shape)
rollout_list = []

(6, 24, 3)


In [36]:
model = model.to('cuda')

In [37]:
initial_positions= torch.from_numpy(initial_positions)

In [38]:
# def rollout(model,initial_positions):
# initial_input=torch.from_numpy(initial_positions)
start = timeit.default_timer()
for i in range(314):
    print(i)
    initial_input = initial_positions.flatten().to('cuda')
    print(initial_input.shape)
    predicitons = model(initial_input.unsqueeze(0))
    predicitons=predicitons.reshape(1,24,3)
    rollout_list.append(predicitons)
    initial_positions= initial_positions[1:,:,:].to('cuda') #get rid of the first the position and add the prediction
    initial_positions = torch.cat([initial_positions, predicitons], dim=0)
stop = timeit.default_timer()
print('Time (with serializing output): ', stop - start)

0
torch.Size([432])
1
torch.Size([432])
2
torch.Size([432])
3
torch.Size([432])
4
torch.Size([432])
5
torch.Size([432])
6
torch.Size([432])
7
torch.Size([432])
8
torch.Size([432])
9
torch.Size([432])
10
torch.Size([432])
11
torch.Size([432])
12
torch.Size([432])
13
torch.Size([432])
14
torch.Size([432])
15
torch.Size([432])
16
torch.Size([432])
17
torch.Size([432])
18
torch.Size([432])
19
torch.Size([432])
20
torch.Size([432])
21
torch.Size([432])
22
torch.Size([432])
23
torch.Size([432])
24
torch.Size([432])
25
torch.Size([432])
26
torch.Size([432])
27
torch.Size([432])
28
torch.Size([432])
29
torch.Size([432])
30
torch.Size([432])
31
torch.Size([432])
32
torch.Size([432])
33
torch.Size([432])
34
torch.Size([432])
35
torch.Size([432])
36
torch.Size([432])
37
torch.Size([432])
38
torch.Size([432])
39
torch.Size([432])
40
torch.Size([432])
41
torch.Size([432])
42
torch.Size([432])
43
torch.Size([432])
44
torch.Size([432])
45
torch.Size([432])
46
torch.Size([432])
47
torch.Size([432])
48

In [21]:
final_rollout = torch.stack(rollout_list)
final_rollout=final_rollout.reshape(314,24,3)
print(final_rollout.shape)

torch.Size([314, 24, 3])


In [22]:
import pickle

In [23]:
with open('rollout_test_0_svd.pkl', 'rb') as f:
    x = pickle.load(f)

In [24]:
val_list=list(x.values())
initial_positions = val_list[0]
predicted_rollout = val_list[1]
ground_truth_rollout = val_list[2]
particle_types = val_list[3]
initial_forces = val_list[4]
predicted_forces = val_list[5]
ground_truth_forces = val_list[6]
global_context = val_list[7]

print(val_list[2])

[[[ 4.0413201e-01  1.8202201e-01  1.6115899e-01]
  [ 3.9859000e-01  2.0020247e-01  1.6115899e-01]
  [ 3.8181570e-01  2.0913921e-01  1.6115899e-01]
  ...
  [ 1.4864514e-02 -1.1183145e-02 -1.4208817e-03]
  [-1.7585070e-03 -1.0133843e-04 -3.6589805e-02]
  [ 2.3985007e-03  4.1560931e-03 -1.2849305e-02]]

 [[ 4.0406400e-01  1.8229400e-01  1.6115899e-01]
  [ 3.9828593e-01  2.0040114e-01  1.6115899e-01]
  [ 3.8139653e-01  2.0911913e-01  1.6115899e-01]
  ...
  [ 1.4865694e-02 -1.1194116e-02 -1.4143625e-03]
  [-1.7587226e-03 -8.6050080e-05 -3.6584064e-02]
  [ 2.3989992e-03  4.1374257e-03 -1.2841122e-02]]

 [[ 4.0399200e-01  1.8256600e-01  1.6115899e-01]
  [ 3.9797893e-01  2.0059700e-01  1.6115899e-01]
  [ 3.8097721e-01  2.0909496e-01  1.6115899e-01]
  ...
  [ 1.4867078e-02 -1.1204292e-02 -1.4085409e-03]
  [-1.7590749e-03 -7.0540911e-05 -3.6577512e-02]
  [ 2.3996395e-03  4.1193389e-03 -1.2833590e-02]]

 ...

 [[ 3.3187401e-01  1.5859200e-01  1.6115899e-01]
  [ 3.4956980e-01  1.5165755e-01  1.611

In [25]:
print(x['predicted_rollout'].shape)

(314, 24, 3)


In [26]:
final_rollout_cpu = final_rollout.detach().cpu()
# convert the tensor to a NumPy array
final_rollout_np = final_rollout_cpu.numpy()

# print the resulting NumPy array
print(final_rollout_np)

[[[ 4.0381873e-01  1.8681490e-01  1.6217260e-01]
  [ 3.9743009e-01  2.0415977e-01  1.6217260e-01]
  [ 3.8044780e-01  2.1170753e-01  1.6217260e-01]
  ...
  [ 6.6827573e-03 -6.7470670e-03 -3.4812454e-03]
  [ 8.9256535e-04 -4.2905286e-04 -4.1223921e-02]
  [-1.1863052e-02  1.9428022e-03 -1.2025378e-02]]

 [[ 4.0475121e-01  1.8734297e-01  1.6217497e-01]
  [ 3.9824441e-01  2.0539109e-01  1.6217497e-01]
  [ 3.8068593e-01  2.1334994e-01  1.6217498e-01]
  ...
  [ 6.9388300e-03 -6.9471821e-03 -3.4848179e-03]
  [ 8.8525331e-04 -4.0322449e-04 -4.1483156e-02]
  [-1.1920440e-02  1.9546682e-03 -1.2111838e-02]]

 [[ 4.0500218e-01  1.8771672e-01  1.6217214e-01]
  [ 3.9839092e-01  2.0599040e-01  1.6217214e-01]
  [ 3.8060200e-01  2.1403374e-01  1.6217215e-01]
  ...
  [ 7.4326321e-03 -7.5074807e-03 -3.5248927e-03]
  [ 8.6032460e-04 -3.2597315e-04 -4.2049326e-02]
  [-1.1919736e-02  1.8653190e-03 -1.2297391e-02]]

 ...

 [[ 3.1185651e-01  2.0194823e-01  1.6110024e-01]
  [ 2.9928362e-01  1.8726707e-01  1.611

In [27]:
print(final_rollout_np.dtype)

float32


In [28]:
x['predicted_rollout']=final_rollout_np

In [29]:
print(x['initial_forces'].shape)

(6, 3)


In [30]:
with open('mlp_rollout_1k.pkl', 'wb') as f:
        pickle.dump(x, f)